In [2]:
import networkx as nx

from src.metro_net import ChengduMetro
from src.utils import read_data

# nodes = read_data("node_info")
# links = read_data("link_info")
# net = ChengduMetro(nodes, links)
# net.print_graph_info()
p, pv = read_data("path"), read_data("pathvia")
p

[INFO] read_data(('path',), {}) executing at 03-31 22:59:25.
[INFO] read_data(('path',), {}) executed in  03-31 22:59:25. (03-31 22:59:25 -> 0.0010s )
[INFO] read_data(('pathvia',), {}) executing at 03-31 22:59:25.
[INFO] read_data(('pathvia',), {}) executed in  03-31 22:59:25. (03-31 22:59:25 -> 0.0000s )


,path_id,length,transfer_cnt,path_str
0,1001100201,1296.0,0,1|1|10
1,1001100301,3280.0,0,1|1|25
2,1001100401,3405.0,0,1|1|26
3,1001100501,1587.0,2,1|1|2_7|-1|6_2|-1|4
4,1001100502,1946.0,2,1|1|5_4|-1|2_2|-1|7
...,...,...,...,...
433,1001113605,2568.5,2,1|1|12_7|1|3_10|1|3
434,1001113606,2633.5,3,1|1|5_4|1|2_3|1|8_10|1|3
435,1001113607,2657.0,3,1|1|2_7|1|1_3|1|12_10|1|3
436,1001113608,2716.0,4,1|1|6_2|-1|3_4|-1|3_7|-1|4_10|1|3


In [3]:
pv

,path_id,node_id1,node_id2,link_type,line,updown
0,1001100201,1001,101200,entry,0,0
1,1001100201,101200,101300,in_vehicle,1,1
2,1001100201,101300,1002,egress,0,0
3,1001100301,1001,101200,entry,0,0
4,1001100301,101200,101470,in_vehicle,1,1
...,...,...,...,...,...,...
3812,1001113609,107461,107401,in_vehicle,7,-1
3813,1001113609,107401,1038,egress,0,0
3814,1001113609,1038,110210,entry,0,0
3815,1001113609,110210,110240,in_vehicle,10,1


In [24]:
import numpy as np
import networkx as nx

G = net.G.copy()
# source, target = np.random.choice(range(1001, 1137), 2)
source, target = 1099, 1045  # 1号线内部换乘
length, sp = nx.single_source_dijkstra(G, source, target)
print(length, sp)
max_length = min(length*1.6, length+ 600)
print(max_length)

1332.0 [1099, 101431, 101421, 101411, 1043, 101410, 101440, 101450, 101460, 101470, 101480, 101490, 101500, 1045]
1932.0


In [35]:
from heapq import heappush

lengths, paths = [length], [sp]
k = 4
G = net.G.copy()

for i in range(1, k):
    for j in range(len(paths[-1]) - 1):
        spur_node = paths[-1][j]
        root_path = paths[-1][:j + 1]
        print(i, j)
        print(spur_node, root_path)



        # remove all nodes connected to spur_node that are in current k-paths
        edges_removed = []
        for c_path in paths:
            if len(c_path) > j and root_path == c_path[:j+1]:
                to_remove_node = c_path[j+1]
                for u, v, edge_attr in [*G.in_edges(nbunch=to_remove_node, data=True), *G.out_edges(nbunch=to_remove_node, data=True)]:
                    edges_removed.append((u,v,edge_attr))
                # if G.has_edge(spur_node, to_remove_node) and G.edges[u, v]['type'] == "in_vehicle":
                #     edge_attr = G.edges[u, v]
                #     G.remove_edge(u, v)
                    # edges_removed.append((u, v, edge_attr))
        print(edges_removed)

        # remove nodes from root path nodes, in/out in_vehicle links
        for n in range(len(root_path) - 1):
            node = root_path[n]
            edges_removed_root = []
            for u, v, edge_attr in [*G.in_edges(nbunch=node, data=True), *G.out_edges(nbunch=node, data=True)]:
                edges_removed_root.append((u, v, edge_attr))
                # print(u, v, edge_attr)
            print(edges_removed_root)
            # G.remove_edges_from(edges_removed_root)

        # spur_paths_length, spur_paths = nx.single_source_dijkstra(G, spur_node, cutoff=max_length)
        # if target in spur_paths and spur_paths[target]:
        #     total_path = root_path[:-1] + spur_paths[target]
        #     total_path_length = ...  # get path length
            # heappush(B, )

    break

1 0
1099 [1099]
[(101430, 101431, {'type': 'platform_swap', 'weight': 3.0, 'line': 0, 'updown': 0}), (1099, 101431, {'type': 'entry', 'weight': 15.0, 'line': 0, 'updown': 0}), (101431, 101421, {'type': 'in_vehicle', 'weight': 147.0, 'line': 1, 'updown': -1}), (101431, 1099, {'type': 'egress', 'weight': 15.0, 'line': 0, 'updown': 0}), (101431, 101430, {'type': 'platform_swap', 'weight': 3.0, 'line': 0, 'updown': 0})]
1 1
101431 [1099, 101431]
[(101420, 101421, {'type': 'platform_swap', 'weight': 3.0, 'line': 0, 'updown': 0}), (101431, 101421, {'type': 'in_vehicle', 'weight': 147.0, 'line': 1, 'updown': -1}), (1123, 101421, {'type': 'entry', 'weight': 15.0, 'line': 0, 'updown': 0}), (101421, 101411, {'type': 'in_vehicle', 'weight': 132.5, 'line': 1, 'updown': -1}), (101421, 1123, {'type': 'egress', 'weight': 15.0, 'line': 0, 'updown': 0}), (101421, 101420, {'type': 'platform_swap', 'weight': 3.0, 'line': 0, 'updown': 0})]
[(101430, 1099, {'type': 'egress', 'weight': 15.0, 'line': 0, 'upd

In [17]:
for i in [*G.in_edges(nbunch=1043, data=True), *G.out_edges(nbunch=1043, data=True)]:
    print(i)

(101410, 1043, {'type': 'egress', 'weight': 15.0, 'line': 0, 'updown': 0})
(101411, 1043, {'type': 'egress', 'weight': 15.0, 'line': 0, 'updown': 0})
(1043, 101410, {'type': 'entry', 'weight': 15.0, 'line': 0, 'updown': 0})
(1043, 101411, {'type': 'entry', 'weight': 15.0, 'line': 0, 'updown': 0})
